Distributed MPC for cooperation scheme using artificial references.
The goal is to implement consensus of a multi-agents system.

$m$ agents with dynamics $x_i(t+1) = f(x_i(t), u_i(t))$.


Implement a sequential scheme as outlined in the corresponding manuscript:

>M. Köhler, M. A. Müller, and F. Allgöwer, "Distributed MPC for Self-Organized Cooperation of Multi-Agent Systems,", 2023, available on arxiv. doi: 10.48550/arXiv.2210.10128

In [ ]:
import mkmpc
import numpy as np
import casadi as cas
import matplotlib.pyplot as plt

We will start with a multi-agent system comprising four to five two-dimensional double integrators.
(We will add the fifth one later to the simulation, but will initialise it now.)
That is $f_i(x_i, u_i) = A_i x_i + B_i u_i$ with

$A_{i} = \begin{bmatrix} 1 & 0 & 1 & 0 \\ 0 & 1 & 0 & 1 \\ 0 & 0 & 1 & 0 \\ 0 & 0 & 0 & 1 \end{bmatrix}$ and $B_{i} = \begin{bmatrix} 0 & 0 \\ 0 & 0 \\ 1 & 0\\ 0 & 1 \end{bmatrix}$.

The output are the first two states, i.e. $y_i = \begin{bmatrix} x_{i,1} \\ x_{i,2} \end{bmatrix}$

In [ ]:
num_agents = 5  # Set number of agents.
state_dims = [4, 4, 4, 4, 4]  # Set the state dimensions.
input_dims = [2, 2, 2, 2, 2]  # Set the input dimensions.
output_dim = 2  # Set the output dimension, which is the same for all agents.

# Specifiy initial states.
initial_state_list = [np.array([[0], [0], [0], [0]]),
                      np.array([[0], [0], [0], [0]]),
                      np.array([[0], [0], [0], [0]]),
                      np.array([[0], [0], [0], [0]])]

dynamics_list = []
output_maps = []
for i in range(num_agents):
    x = cas.MX.sym('x', state_dims[i])  # Create symbolic for state.
    u = cas.MX.sym('u', input_dims[i])  # Create symbolic for input.
    A = np.array([[1, 0, 1, 0],
                  [0, 1, 0, 1],
                  [0, 0, 1, 0],
                  [0, 0, 0, 1]])
    B = np.array([[0, 0],
                  [0, 0],
                  [1, 0],
                  [0, 1]])
    # Create state dynamics as a casadi function.
    dynamics_list.append(cas.Function('dynamics', [x,u], [A@x + B@u], ['x', 'u'], ['f']))
    # Create output map as a casadi function.
    C = np.array([[1, 0, 0, 0],
                  [0, 1, 0, 0]])
    output_maps.append(cas.Function('output', [x,u], [C@x], ['x', 'u'], ['h']))
    
# Define the agents.
agents = []
for i in range(num_agents):
    # Initialise the agent.
    agents.append(mkmpc.agent(id = i+1, state_dim = state_dims[i], input_dim = input_dims[i],
                              dynamics = dynamics_list[i], initial_time=0, initial_state=initial_state_list[0],
                              output_map=output_maps[i], output_dim=output_dim))


Set constraints for each agent and visualise them.

For simplicity, we use polytopes for the first two states.
We use vertex notation for plotting and half-space notation for the optimization.

- $\mathbb{X}_1 = \mathrm{co} \{ \begin{bmatrix} 1.1 \\ -2.1 \end{bmatrix}, \begin{bmatrix} 1.1 \\ 4.1 \end{bmatrix} , \begin{bmatrix} -1.1 \\ 4.1 \end{bmatrix} , \begin{bmatrix} -1.1 \\ -2.1 \end{bmatrix}  \} = \{ \begin{bmatrix} x_1 \\ x_2 \end{bmatrix} \mid \begin{bmatrix} 0 & -1 \\ 1 & 0 \\ 0 & 1 \\ -1 & 0 \end{bmatrix} \begin{bmatrix} x_1 \\ x_2 \end{bmatrix} \le \begin{bmatrix}  2.1 \\ 1.1 \\ 4.1 \\ 1.1 \end{bmatrix}$
- $\mathbb{X}_2 = \mathbb{X}_3 = \mathrm{co} \{ \begin{bmatrix} 4.1 \\ -2.1 \end{bmatrix}, \begin{bmatrix} 4.1 \\ 2.1 \end{bmatrix} , \begin{bmatrix} -1.1 \\ 2.1 \end{bmatrix} , \begin{bmatrix} -1.1 \\ -2.1 \end{bmatrix}  \} = \{ \begin{bmatrix} x_1 \\ x_2 \end{bmatrix} \mid \begin{bmatrix} -1 & 0 \\ 1 & 0 \\ 0 & -1 \\ 0 & 1 \end{bmatrix} \begin{bmatrix} x_1 \\ x_2 \end{bmatrix} \le \begin{bmatrix}  1.1 \\ 4.1 \\ 2.1 \\ 2.1 \end{bmatrix}$
- $\mathbb{X}_4 = \mathbb{X}_5 = \mathrm{co} \{ \begin{bmatrix} 3.1 \\ 0 \end{bmatrix}, \begin{bmatrix} 0 \\ 3.1 \end{bmatrix} , \begin{bmatrix} -3.1 \\ 0 \end{bmatrix} , \begin{bmatrix} 0 \\ -3.1 \end{bmatrix}  \} = \{ \begin{bmatrix} x_1 \\ x_2 \end{bmatrix} \mid \begin{bmatrix} 1 & -1 \\ 1 & 1 \\ -1 & 1 \\ -1 & -1 \end{bmatrix} \begin{bmatrix} x_1 \\ x_2 \end{bmatrix} \le \begin{bmatrix}  3.1 \\ 3.1 \\ 3.1 \\ 3.1 \end{bmatrix} $

We constrain the third and fourth state as well as the inputs between $-0.25$ and $0.25$ for all agents.

In [ ]:
# Set constraints for each agent.
box_input_constr = np.array([[-0.25, 0.25]])  # Define input constraints for all agents.

agents[0].set_constraints(box_input_constraints=box_input_constr)  # Set input constraints.
# Set state constraints:
agents[0].state_constraints["A"] = np.array([[0, -1, 0, 0],
                                             [1, 0, 0, 0],
                                             [0, 1, 0, 0],
                                             [-1, 0, 0, 0],
                                             [0, 0, 1, 0],  # Constraints for 3rd and 4th state from here.
                                             [0, 0, -1, 0],
                                             [0, 0, 0, 1],
                                             [0, 0, 0, -1]
                                            ]) 
agents[0].state_constraints["b"] = np.array([[2.1], [1.1], [4.1], [1.1], [0.25], [0.25], [0.25], [0.25]])

agents[1].set_constraints(box_input_constraints=box_input_constr)  # Set input constraints.
# Set state constraints:
agents[1].state_constraints["A"] = np.array([[-1, 0, 0, 0],
                                             [1, 0, 0, 0],
                                             [0, -1, 0, 0],
                                             [0, 1, 0, 0],
                                             [0, 0, 1, 0],  # Constraints for 3rd and 4th state from here.
                                             [0, 0, -1, 0],
                                             [0, 0, 0, 1],
                                             [0, 0, 0, -1]
                                            ]) 
agents[1].state_constraints["b"] = np.array([[1.1], [4.1], [2.1], [2.1], [0.25], [0.25], [0.25], [0.25]])

agents[2].set_constraints(box_input_constraints=box_input_constr)  # Set input constraints.
# Set state constraints:
agents[2].state_constraints["A"] = np.array([[-1, 0, 0, 0],
                                             [1, 0, 0, 0],
                                             [0, -1, 0, 0],
                                             [0, 1, 0, 0],
                                             [0, 0, 1, 0],  # Constraints for 3rd and 4th state from here.
                                             [0, 0, -1, 0],
                                             [0, 0, 0, 1],
                                             [0, 0, 0, -1]
                                            ]) 
agents[2].state_constraints["b"] = np.array([[1.1], [4.1], [2.1], [2.1], [0.25], [0.25], [0.25], [0.25]])

agents[3].set_constraints(box_input_constraints=box_input_constr)  # Set input constraints.
# Set state constraints:
agents[3].state_constraints["A"] = np.array([[1, -1, 0, 0],
                                             [1, 1, 0, 0],
                                             [-1, 1, 0, 0],
                                             [-1, -1, 0, 0],
                                             [0, 0, 1, 0],  # Constraints for 3rd and 4th state from here.
                                             [0, 0, -1, 0],
                                             [0, 0, 0, 1],
                                             [0, 0, 0, -1]
                                            ]) 
agents[3].state_constraints["b"] = np.array([[3.1], [3.1], [3.1], [3.1], [0.25], [0.25], [0.25], [0.25]])

agents[4].set_constraints(box_input_constraints=box_input_constr)  # Set input constraints.
# Set state constraints:
agents[4].state_constraints["A"] = np.array([[1, -1, 0, 0],
                                                  [1, 1, 0, 0],
                                                  [-1, 1, 0, 0],
                                                  [-1, -1, 0, 0],
                                                  [0, 0, 1, 0],  # Constraints for 3rd and 4th state from here.
                                                  [0, 0, -1, 0],
                                                  [0, 0, 0, 1],
                                                  [0, 0, 0, -1]
                                                 ])  
agents[4].state_constraints["b"] = np.array([[3.1], [3.1], [3.1], [3.1], [0.25], [0.25], [0.25], [0.25]])

## Visualise constraints on the first two states. For simplicity, use the vertices.
# Note that the inequality constraints above need to correspond to the following vertices.

agents[0].state_constraints_for_plot = np.array([[1.1,1.1,-1.1,-1.1],[-2.1,4.1,4.1,-2.1]])
agents[1].state_constraints_for_plot = np.array([[4.1,4.1,-1.1,-1.1],[-2.1,2.1,2.1,-2.1]])
agents[2].state_constraints_for_plot = np.array([[4.1,4.1,-1.1,-1.1],[-2.1,2.1,2.1,-2.1]])
agents[3].state_constraints_for_plot = np.array([[3.1,0,-3.1,0],[0,3.1,0,-3.1]])
agents[4].state_constraints_for_plot = np.array([[3.1,0,-3.1,0],[0,3.1,0,-3.1]])

fig, ax = plt.subplots()
ax.grid(True, which='both')  # Draw grid lines.
ax.axis('equal')  # Make plot a box.
ax.fill(agents[0].state_constraints_for_plot[0,:], agents[0].state_constraints_for_plot[1,:],
         edgecolor='black', linewidth=2,
         facecolor=(0,0,0.9,0.5))  # Draw polyhedra.
ax.fill(agents[1].state_constraints_for_plot[0,:], agents[1].state_constraints_for_plot[1,:],
         edgecolor='black', linewidth=2,
         facecolor=(0,0.9,0,0.5))  # Draw polyhedra.
ax.fill(agents[3].state_constraints_for_plot[0,:], agents[3].state_constraints_for_plot[1,:],
         edgecolor='black', linewidth=2,
         facecolor=(0.9,0,0,0.5))  # Draw polyhedra.
plt.show()



Set neighbours as
* $\mathcal{N}_1 = \{2, 4\}$
* $\mathcal{N}_2 = \{1\}$
* $\mathcal{N}_3 = \{4\}$
* $\mathcal{N}_4 = \{3, 1\}$

The graph changes when the fifth agents joins.

In [ ]:
# Set neighbours in lists as described above. Note that the index starts at 0.
agents[0].neighbours = [agents[1], agents[3]]
agents[1].neighbours = [agents[0]]
agents[2].neighbours = [agents[3]]
agents[3].neighbours = [agents[0], agents[2]]

Define the stage cost for tracking, i.e. $l_i = \Vert x_i - x_{\mathrm{c},i} \Vert^2_{Q_i} + \Vert u_i - u_{\mathrm{c},i} \Vert^2_{R_i}$.

In [ ]:
for i in range(num_agents):
    # Define the artificial equilibrium. 
    x = cas.MX.sym('x', state_dims[i])
    u = cas.MX.sym('u', input_dims[i])
    xc = cas.MX.sym('x_c', state_dims[i])
    uc = cas.MX.sym('u_c', input_dims[i])
    
    # Set the weight for the distance of the state to the equilibrium.
    Q = np.eye(state_dims[i])
    # Set the weight for the distance of the input to the equilibrium.
    R = np.eye(input_dims[i])
    
    stage_cost = cas.Function('stage_cost', [x, u, xc, uc], [ (x - xc).T@Q@(x - xc) + (u - uc).T@R@(u - uc) ],
                              ['x', 'u', 'xc', 'uc'], ['l'])
    
    # Add stage cost to agents.
    agents[i].stage_cost = stage_cost

Define the cost for cooperation.

We want to have consensus in the outputs.
As the bilateral costs we take $V_{ij}^\mathrm{c} = \Vert y_{\mathrm{c},i} - y_{\mathrm{c},j} \Vert^2$.

In [ ]:
yc1 = cas.MX.sym('yc1', output_dim)
yc2 = cas.MX.sym('yc2', output_dim)
bilat_coop_cost = cas.Function('cooperation_cost', [yc1, yc2], [ (yc1 - yc2).T@(yc1 - yc2)], ['yc1', 'yc2'], ['V_ij^c'])
# Set the bilateral cooperation cost for each agent.
for agent in agents:
    agent.bilat_coop_cost = bilat_coop_cost

Define the set of admissible cooperation outputs $\mathcal{Y}_i$ to be almost the same as the state constraint sets for the first two states.

Define the sets again for computations in half-space form and for plotting in vertex form.


- $\mathcal{Y}_1 = \mathrm{co} \{ \begin{bmatrix} 1 \\ -2 \end{bmatrix}, \begin{bmatrix} 1 \\ 4 \end{bmatrix} , \begin{bmatrix} -1 \\ 4 \end{bmatrix} , \begin{bmatrix} -1 \\ -2 \end{bmatrix}  \} = \{ \begin{bmatrix} x_1 \\ x_2 \end{bmatrix} \mid \begin{bmatrix} 0 & -1 \\ 1 & 0 \\ 0 & 1 \\ -1 & 0 \end{bmatrix} \begin{bmatrix} x_1 \\ x_2 \end{bmatrix} \le \begin{bmatrix}  2 \\ 1 \\ 4 \\ 1 \end{bmatrix}$
- $\mathcal{Y}_2 = \mathcal{Y}_3 = \mathrm{co} \{ \begin{bmatrix} 4 \\ -2 \end{bmatrix}, \begin{bmatrix} 4 \\ 2 \end{bmatrix} , \begin{bmatrix} -1 \\ 2 \end{bmatrix} , \begin{bmatrix} -1 \\ -2 \end{bmatrix}  \} = \{ \begin{bmatrix} x_1 \\ x_2 \end{bmatrix} \mid \begin{bmatrix} -1 & 0 \\ 1 & 0 \\ 0 & -1 \\ 0 & 1 \end{bmatrix} \begin{bmatrix} x_1 \\ x_2 \end{bmatrix} \le \begin{bmatrix}  1 \\ 4 \\ 2 \\ 2 \end{bmatrix}$
- $\mathcal{Y}_4 = \mathcal{Y}_5 = \mathrm{co} \{ \begin{bmatrix} 3 \\ 0 \end{bmatrix}, \begin{bmatrix} 0 \\ 3 \end{bmatrix} , \begin{bmatrix} -3 \\ 0 \end{bmatrix} , \begin{bmatrix} 0 \\ -3 \end{bmatrix}  \} = \{ \begin{bmatrix} x_1 \\ x_2 \end{bmatrix} \mid \begin{bmatrix} 1 & -1 \\ 1 & 1 \\ -1 & 1 \\ -1 & -1 \end{bmatrix} \begin{bmatrix} x_1 \\ x_2 \end{bmatrix} \le \begin{bmatrix}  3 \\ 3 \\ 3 \\ 3 \end{bmatrix} $

In [ ]:
# Add a new attribute to the agents, a dictionary that contains the above matrices. 

agents[0].cooperation_output_constraint = {}
agents[0].cooperation_output_constraint["A"] = np.array([[0, -1],
                                                  [1, 0],
                                                  [0, 1],
                                                  [-1, 0]
                                                 ]) 
agents[0].cooperation_output_constraint["b"] = np.array([[2], [1], [4], [1]])

agents[1].cooperation_output_constraint = {}
agents[1].cooperation_output_constraint["A"] = np.array([[-1, 0],
                                                  [1, 0],
                                                  [0, -1],
                                                  [0, 1]
                                                 ]) 
agents[1].cooperation_output_constraint["b"] = np.array([[1], [4], [2], [2]])

agents[2].cooperation_output_constraint = {}
agents[2].cooperation_output_constraint["A"] = np.array([[-1, 0],
                                                  [1, 0],
                                                  [0, -1],
                                                  [0, 1]
                                                 ]) 
agents[2].cooperation_output_constraint["b"] = np.array([[1], [4], [2], [2]])

agents[3].cooperation_output_constraint = {}
agents[3].cooperation_output_constraint["A"] = np.array([[1, -1],
                                                  [1, 1],
                                                  [-1, 1],
                                                  [-1, -1]
                                                 ]) 
agents[3].cooperation_output_constraint["b"] = np.array([[3], [3], [3], [3]])

agents[4].cooperation_output_constraint = {}
agents[4].cooperation_output_constraint["A"] = np.array([[1, -1],
                                                  [1, 1],
                                                  [-1, 1],
                                                  [-1, -1]
                                                 ])  
agents[4].cooperation_output_constraint["b"] = np.array([[3], [3], [3], [3]])

## Visualise constraints on the first two states. For simplicity, use the vertices.
# Note that the inequality constraints above need to correspond to the following vertices.

agents[0].cooperation_output_constraint["vertex_mat"] = np.array([[1,1,-1,-1],[-2,4,4,-2]])
agents[1].cooperation_output_constraint["vertex_mat"] = np.array([[4,4,-1,-1],[-2,2,2,-2]])
agents[2].cooperation_output_constraint["vertex_mat"] = np.array([[4,4,-1,-1],[-2,2,2,-2]])
agents[3].cooperation_output_constraint["vertex_mat"] = np.array([[3,0,-3,0],[0,3,0,-3]])
agents[4].cooperation_output_constraint["vertex_mat"] = np.array([[3,0,-3,0],[0,3,0,-3]])

fig, ax = plt.subplots()
ax.grid(True, which='both')  # Draw grid lines.
ax.axis('equal')  # Make plot a box.
ax.fill(agents[0].cooperation_output_constraint["vertex_mat"][0,:], agents[0].cooperation_output_constraint["vertex_mat"][1,:],
         edgecolor='black', linewidth=2,
         facecolor=(0,0,0.9,0.5))  # Draw polyhedra.
ax.fill(agents[1].cooperation_output_constraint["vertex_mat"][0,:], agents[1].cooperation_output_constraint["vertex_mat"][1,:],
         edgecolor='black', linewidth=2,
         facecolor=(0,0.9,0,0.5))  # Draw polyhedra.
ax.fill(agents[3].cooperation_output_constraint["vertex_mat"][0,:], agents[3].cooperation_output_constraint["vertex_mat"][1,:],
         edgecolor='black', linewidth=2,
         facecolor=(0.9,0,0,0.5))  # Draw polyhedra.
plt.show()


# Set up the sequential MPC scheme.

### Set simulation and MPC parameters.

In [ ]:
last_simulation_time = 40
horizon = 5

### Initialisation of all agents:

In [ ]:
# Set all cooperation outputs to zero for the initialisation since 0 is an admissible cooperation output for all agents.
for agent in agents:
    agent.current_cooperation_output = np.zeros((agent.output_dim, 1))

# Set initial states.
agents[0].current_state = np.array([[-1], [4], [0], [0]])
agents[1].current_state = np.array([[2], [1.8], [0], [0]])
agents[2].current_state = np.array([[3], [-1.5], [0], [0]])
agents[3].current_state = np.array([[-2], [0], [0], [0]])
agents[4].current_state = np.array([[0], [-2], [0], [0]])

# Solve the optimisation problem for all agents.
for agent in agents:
    agent.current_MPC_sol = mkmpc.MPC_for_cooperation(agent, horizon=horizon)
    
# Set resulting optimal cooperation output as initial cooperation output.
for agent in agents:
    agent.current_cooperation_output = agent.current_MPC_sol["yc_opt"]

# Set initial state as current cooperation output.
for agent in agents:
    agent.current_cooperation_output = agent.current_state[0:2]
    


In [ ]:
# Plot the initial solution.

fig, ax = plt.subplots()
for agent in agents:
    #fig, ax = plt.subplots()  # Uncomment to show individual plots.
    time_steps = range(0, horizon+1)
    label_str = "agent " + agent.id  # For labelling the plot.
    ax.plot(agent.current_MPC_sol["x_opt"][0,:], agent.current_MPC_sol["x_opt"][1,:], label=label_str, marker='x', markersize=5)
    ax.grid(True, which='both')

### MPC algorithm

In [ ]:
# Keep track of the closed-loop system.
closed_loop_evolution = []
time_step_for_change = 19  # Set the time step where the system changes.

# Create a list containing the agents that are included in the multi-agent system.
agents_in_system = []
for i in [0,1,2,3]:
    agents_in_system.append(agents[i])

for t in range(0, last_simulation_time + 1):
    # Track the time.
    closed_loop_evolution.append([None]*len(agents))
    
    # Go in sequence over the agents.
    for agent in agents_in_system:
        closed_loop_evolution[t][agents.index(agent)] = {"time":t}
        agent.current_time = t    
        
        # Keep track of the current state.
        closed_loop_evolution[t][agents.index(agent)].update({"current_state":agent.current_state})
        
        # Solve the MPC problem.
        agent.current_MPC_sol = mkmpc.MPC_for_cooperation(agent, horizon=horizon)
        # Keep track of the solution.
        closed_loop_evolution[t][agents.index(agent)].update(agent.current_MPC_sol)
        # Update the current state of the agent. 
        # Without model errors and satisfaction of the dynamic constraint, the next predicted state will be the next closed-loop state.
        agent.current_state = agent.current_MPC_sol["x_opt"][0:agent.state_dim, 1:2]
        # Update the cooperation output of the agent.
        agent.current_cooperation_output = agent.current_MPC_sol["yc_opt"]
            
    # For plotting, pad the times agents were inactive.
    agents_not_in_system = [agent for agent in agents if agent not in agents_in_system]
    for agent in agents_not_in_system:
        closed_loop_evolution[t][agents.index(agent)] = {"time":t}
        agent.current_time = t  # Update the agents time.
        closed_loop_evolution[t][agents.index(agent)].update({"current_state":agent.current_state})  # Save the current (unchanged) state.
    
    # Change the system at a specified timestep.
    if t == time_step_for_change:
        # Add agent to optimisation sequence.
        agents_in_system.append(agents[4])
        # Update neighbours.
        agents[0].neighbours = [agents[3]]
        agents[1].neighbours = [agents[2]]
        agents[2].neighbours = [agents[1], agents[3], agents[4]]
        agents[3].neighbours = [agents[0], agents[2], agents[4]]
        agents[4].neighbours = [agents[3], agents[2]]      


# Plots
#### Plot the closed-loop evolution.
Note that the figures in the paper were created using a different work flow.

In [ ]:
time_steps = range(0, last_simulation_time+1)

fig1, ax1 = plt.subplots()
ax1.grid(True)
fig2, ax2 = plt.subplots()
ax2.grid(True)
fig3, ax3 = plt.subplots()
ax3.grid(True)

line_styles = ['solid', 'dotted', 'dashdot', 'dashed', (0, (3, 1, 1, 1, 1, 1))]
colour_styles = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple']

for agent in agents:
    # Extract state evolution of agent.
    agent_state_evo = []
    for t in time_steps:
        agent_state_evo.append(closed_loop_evolution[t][agents.index(agent)]["current_state"])
    # Build state evolution matrix.
    state_evo_mat = np.concatenate(agent_state_evo, axis=1)
    
    label_str = "Agent " + agent.id  # For labelling the plot.
    
    # Plot evolution of first states.
    ax1.plot(time_steps, state_evo_mat[0, :], label=label_str, linestyle=line_styles[agents.index(agent)], linewidth=2)
    ax1.minorticks_on()
    ax1.legend()
    
    # Plot evolution of second states.
    ax2.plot(time_steps, state_evo_mat[1, :], label=label_str, linestyle=line_styles[agents.index(agent)], linewidth=2)
    ax2.minorticks_on()
    ax2.legend()
    
    # Set line width of dotted trajectory to be larger in the last plot.
    if line_styles[agents.index(agent)] == 'dotted':
        line_width = 3
    else:
        line_width = 2
        
    # Plot 2D evolution.    
    ax3.plot(state_evo_mat[0, :], state_evo_mat[1, :], label=label_str, linestyle=line_styles[agents.index(agent)], linewidth=line_width, color=colour_styles[agents.index(agent)])
    # Plot last point.
    ax3.plot(state_evo_mat[0, -1], state_evo_mat[1, -1], linestyle=line_styles[agents.index(agent)], linewidth=1, marker='x', color='k')
#    ax3.minorticks_on()
    ax3.legend()
    
# Add the constraint sets.
ax3.axis('equal')  # Make plot a box.
ax3.fill(agents[0].cooperation_output_constraint["vertex_mat"][0,:], agents[0].cooperation_output_constraint["vertex_mat"][1,:],
         edgecolor='black', linewidth=2,
         facecolor=(0.1,0.1,0.1,0.05))  # Draw polyhedra.
ax3.fill(agents[1].cooperation_output_constraint["vertex_mat"][0,:], agents[1].cooperation_output_constraint["vertex_mat"][1,:],
         edgecolor='black', linewidth=2,
         facecolor=(0.1,0.1,0.1,0.05))  # Draw polyhedra.
ax3.fill(agents[3].cooperation_output_constraint["vertex_mat"][0,:], agents[3].cooperation_output_constraint["vertex_mat"][1,:],
         edgecolor='black', linewidth=2,
         facecolor=(0.1,0.1,0.1,0.05))  # Draw polyhedra.
ax3.grid(True, which='both')  # Draw grid lines.
ax3.set_axisbelow(True)

# Set axis labels.
ax1.set_xlabel('time steps')
ax1.set_ylabel('x_{i,1}')

ax2.set_xlabel('time steps')
ax2.set_ylabel('x_{i,2}')

ax3.set_xlabel('x_{i,1}')
ax3.set_ylabel('x_{i,2}')
